In [ ]:
import crestdsl.model as crest
import crestdsl.ui as crestui

import logging
sslog = logging.getLogger("crestdsl.verification.statespace")  # show us some logging info
sslog.setLevel(logging.INFO)

# Model the germination system

In [ ]:
# resources
onOff = crest.Resource(unit="onOff", domain=["on", "off"])
celsius = crest.Resource(unit="Celsius", domain=crest.REAL)
time = crest.Resource(unit="Time", domain=crest.REAL)

In [ ]:
class GerminationBox(crest.Entity):
    switch = crest.Input(resource=onOff, value="on")   
    temperature = crest.Local(resource=celsius, value=22)
    
    state = current = crest.State()
    
    @crest.update(state=state, target=temperature)
    def update_temp(self, dt):
        if self.switch.value == "on":
            # don't exceed 40 (this is the maximum temperature)
            return min(40, self.temperature.value + 0.5 * dt)
        else:
            # don't go below 22 (minimum = current room temperature)
            return max(22, self.temperature.value - 0.25 * dt)
        
crestui.plot(GerminationBox())

In [ ]:
class GerminationSystem(crest.Entity):
    switch = crest.Input(resource=onOff, value="on")
    countdown = crest.Local(resource=time, value=0)
    
    germinationbox_one = GerminationBox()
    germinationbox_two = GerminationBox()
    
    off = current = crest.State()
    on = crest.State()
    pause = crest.State()
    boxOne = crest.State()
    boxTwo = crest.State()
    
    # transitions
    @crest.transition(source=[on,boxOne,boxTwo,pause], target=off)
    def on_to_off(self):
        return self.switch.value == "off"
    
    @crest.transition(source=off, target=on)
    def off_to_on(self):
        return self.switch.value == "on"
    
    @crest.transition(source=[boxOne,boxTwo,pause], target=on)
    def return_to_on(self):
        return self.countdown.value <= 0
    
    @crest.transition(source=on, target=[boxOne,boxTwo])
    def on_to_box(self):
        return self.countdown.value <= 0
    
    @crest.transition(source=on, target=pause)
    def on_to_pause(self):
        return self.countdown.value <= 0
    
    """ updates """
    
    @crest.update(state=[on,boxOne,boxTwo,pause], target=countdown)
    def reduce_countdown(self, dt):
        return max(0, self.countdown.value - dt)
    
    @crest.update(state=boxOne, target=germinationbox_one.switch)
    def turn_on_boxOne(self, dt):
        return "on"
    
    @crest.update(state=[on,pause,off,boxTwo], target=germinationbox_one.switch)
    def turn_off_boxOne(self, dt):
        return "off"
    
    @crest.update(state=boxTwo, target=germinationbox_two.switch)
    def turn_on_boxTwo(self, dt):
        return "on"
    
    @crest.update(state=[on,pause,off,boxOne], target=germinationbox_two.switch)
    def turn_off_boxTwo(self, dt):
        return "off"
    
    """ transition actions """
    
    @crest.action(transition=off_to_on, target=countdown)
    def set_countdown_zero_when_turn_on(self):
        return 0
    
    @crest.action(transition=on_to_box, target=countdown)
    def set_countdown_thirty(self):
        return 30
    
    @crest.action(transition=on_to_pause, target=countdown)
    def set_countdown_ten(self):
        return 10
    
crestui.plot(GerminationSystem())

Create a model instance of the system:

In [ ]:
system = GerminationSystem()

# Simple (Convenience) API

## Define Checks

In [ ]:
from crestdsl.verification import check
system = GerminationSystem()  # initialise system

statecheck = check(system) == system.off  # create a state check 
print(statecheck.check())  # trigger the check (returns True)

portcheck = check(system.countdown) >= 0  # two port checks
another = check(system.germinationbox_one.temperature) == 33

# use of operators to alter / combine checks
negate =  --another           # negation using minus
orcheck = another | negate    # disjunction using | 
andcheck = another & negate   # conjunction using &

# compare two ports' values
port_compare = check(system.germinationbox_one.temperature) > system.germinationbox_two.temperature

## Verification - convenience API 

In [ ]:
from crestdsl.verification import check, is_possible, before

chk = check(system.countdown) == 0

# usage example: chaining formula specification, after and before 
is_possible(chk).after(20).check()  # exploration takes ~40 seconds
before(100).always(chk).check()  # takes 1min 20secs

# TCTL API

## Examples of TCTL formulas

In [ ]:
from crestdsl.verification import tctl, check

# define a system and checks
sys = GerminationSystem()
tl = check(sys.countdown) < 5
tvl = check(sys.countdown) <= 3
is_pause = check(sys) == sys.pause


tl_until_tvl = tctl.AU(tl, tvl)  # default interval = [0, inf)

# reachability formula
tvl_reachable_before_30 = tctl.EF(tvl, tctl.Interval(end=30))

# use of operators for interval specifications
after_10 = tctl.Interval() > 10
within_30 = tctl.Interval() <= 30
always_pause_within_30 = tctl.AG(tctl.EF(is_pause, within_30), after_10)

## TCTL Model Checking

In [ ]:
from crestdsl.verification import StateSpace, ModelChecker
sys = GerminationSystem()
formula = tctl.AG(tctl.EF(is_pause, tctl.Interval() > 10), 
                    tctl.Interval() <= 30)
statespace = StateSpace(sys)
statespace.explore(None)      # explore entire state space (takes ~20 seconds)
mc = ModelChecker(statespace)
mc.check(tvl_reachable_before_30)  # trigger model checking